In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from scipy import stats

# Взаимосвязь между номинативными зависимыми переменными которые имеют только две градации

df = pd.read_csv('https://stepik.org/media/attachments/course/524/train.csv')
glm_binom2 = smf.glm(formula="Survived ~ C(Sex)",
                     data=df, family=sm.families.Binomial())

df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [2]:
model = glm_binom2.fit()
model.summary()
pd.crosstab(df.Survived, df.Sex)
log_odds_male = np.log(109 / 468)
log_odds_female = np.log(233 / 81) # intercept
print(f'log_odds_male: {log_odds_male}')
print(f'log_odds_female: {log_odds_female}')
odds_male = 109/468
odds_female = 233/81
odds_ratio = np.log(odds_male / odds_female) # coef_
print(f'odds_male / odds_female: {odds_ratio}')

# sklearn
Y = df.Survived
X = pd.get_dummies(df.Sex, drop_first=True)
model = LogisticRegression()
model.fit(X,Y)
result = pd.DataFrame(data=[np.ndarray.item(model.intercept_),np.ndarray.item(model.coef_)],
             index=['intercept','coef'], columns=['coefs'])

'''Необходим дисперсионный анализ полученных коэффициентов (через нормальное распределение) Но: коэффициент (intercept или coef) равен 0 в генеральной совокупности.'''

# выживаемость по полу
glm_binom = smf.glm(
    formula="Survived ~ C(Sex)"
    , data=df
    , family=sm.families.Binomial()).fit()

print()
print(f'Показатель остатков нашей модели: {glm_binom.aic}') #  Чем меньше тем лучше. Неодбходимо сравнить остатки с моделью в котором только интерсепт
print()

result

log_odds_male: -1.457120413688503
log_odds_female: 1.0565892988932617
odds_male / odds_female: -2.5137097125817647

Показатель остатков нашей модели: 921.8039249773079



,coefs
intercept,1.016937
coef,-2.446469


In [3]:
# выживаемость по классам
glm_binom = smf.glm(
    formula="Survived ~ C(Pclass)"
    , data=df
    , family=sm.families.Binomial()).fit()
print(glm_binom.aic)

1089.1080125305389


In [4]:
# Логистическая модель с двумя предикторами

pd.crosstab(df.query('Sex == "female"').Survived, df.query('Sex == "female"').Pclass) # Для женщин
pd.crosstab(df.query('Sex == "male"').Survived, df.query('Sex == "male"').Pclass) # для мужчин

Pclass,1,2,3
Survived,,,
0,77,91,300
1,45,17,47
